Import the necessary libraries.
Load the we_fn_use_c_marketing_customer_value_analysis.csv into the variable customer_df.
First look at its main features (head, shape, info).
Rename the columns so they follow the PE8 (snake case).
Change effective to date column to datetime format.
Check NaN values per column.
Define a function that differentiates between continuous and discrete variables. Hint: A number of unique values might be useful. Store continuous data into a continuous variable and do the same for discrete and categorical.
For the categorical data, check if there is some kind of text in a variable so we would need to clean it. Hint: Use the same method you used in step 7. Depending on the implementation, decide what to do with the variables you get.
Get categorical features.
What should we do with the customer_id column?

In [11]:
#Import the necessary libraries.

import numpy as np
import scipy.stats as stats
from scipy.stats import norm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import regplot
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Load the we_fn_use_c_marketing_customer_value_analysis.csv into the variable customer_df.
customer_df = pd.read_csv("../files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv")

In [5]:
#First look at its main features (head, shape, info).
customer_df.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [6]:
customer_df.shape

(9134, 24)

In [8]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer                       9134 non-null   object 
 1   State                          9134 non-null   object 
 2   Customer Lifetime Value        9134 non-null   float64
 3   Response                       9134 non-null   object 
 4   Coverage                       9134 non-null   object 
 5   Education                      9134 non-null   object 
 6   Effective To Date              9134 non-null   object 
 7   EmploymentStatus               9134 non-null   object 
 8   Gender                         9134 non-null   object 
 9   Income                         9134 non-null   int64  
 10  Location Code                  9134 non-null   object 
 11  Marital Status                 9134 non-null   object 
 12  Monthly Premium Auto           9134 non-null   i

In [9]:
#Rename the columns so they follow the PE8 (snake case).
standard_columns = [customer_df.columns[i].lower() for i in range(len(customer_df.columns))]
standard_columns = [col_name.lower().replace(' ', '_') for col_name in customer_df.columns]
customer_df.columns = standard_columns

In [10]:
customer_df.columns

Index(['customer', 'state', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'employmentstatus', 'gender',
       'income', 'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount',
       'vehicle_class', 'vehicle_size'],
      dtype='object')

In [17]:
# Change effective to date column to datetime format.
customer_df['effective_to_date'] = pd.to_datetime(customer_df['effective_to_date'])
customer_df['effective_to_date']

0      2011-02-24
1      2011-01-31
2      2011-02-19
3      2011-01-20
4      2011-02-03
          ...    
9129   2011-02-10
9130   2011-02-12
9131   2011-02-06
9132   2011-02-03
9133   2011-02-14
Name: effective_to_date, Length: 9134, dtype: datetime64[ns]

In [18]:
# Check NaN values per column.
customer_df.isna().sum()

customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
dtype: int64

In [94]:
# Define a function that differentiates between continuous and discrete variables. Hint: A number of unique values might be useful. 
# Store continuous data into a continuous variable and do the same for discrete and categorical.
#data = customer_df.copy()
def separate_variables(df):

    continuous_columns = []
    discrete_columns = []
    thresh_hold = 100

    for i in df.columns:
        if df[i].nunique() >= thresh_hold:
            continuous_columns.append(i)
        else:
            discrete_columns.append(i)
    continuous_variables = df[continuous_columns]
    discrete_variables = df[discrete_columns]
    return discrete_variables, continuous_variables


In [95]:
discrete, continuous   = separate_variables(data)

In [118]:
discrete.dtypes

state                                object
response                             object
coverage                             object
education                            object
effective_to_date            datetime64[ns]
employmentstatus                     object
gender                               object
location_code                        object
marital_status                       object
months_since_last_claim               int64
number_of_open_complaints             int64
number_of_policies                    int64
policy_type                          object
policy                               object
renew_offer_type                     object
sales_channel                        object
vehicle_class                        object
vehicle_size                         object
dtype: object

In [103]:
# For the categorical data, check if there is some kind of text in a variable so we would need to clean it. 
# Hint: Use the same method you used in step 7. 
# Depending on the implementation, decide what to do with the variables you get.
for i in discrete:
    print(discrete[i].value_counts())

California    3150
Oregon        2601
Arizona       1703
Nevada         882
Washington     798
Name: state, dtype: int64
No     7826
Yes    1308
Name: response, dtype: int64
Basic       5568
Extended    2742
Premium      824
Name: coverage, dtype: int64
Bachelor                2748
College                 2681
High School or Below    2622
Master                   741
Doctor                   342
Name: education, dtype: int64
2011-01-10    195
2011-01-27    194
2011-02-14    186
2011-01-26    181
2011-01-17    180
2011-01-19    179
2011-01-03    178
2011-01-31    178
2011-01-20    173
2011-02-26    169
2011-01-28    169
2011-02-19    168
2011-02-27    167
2011-01-05    167
2011-01-11    166
2011-02-04    164
2011-02-28    161
2011-02-10    161
2011-01-21    160
2011-01-29    160
2011-01-02    160
2011-02-03    158
2011-02-22    158
2011-02-05    158
2011-02-07    157
2011-02-12    156
2011-01-23    155
2011-01-18    154
2011-02-01    154
2011-01-15    153
2011-01-14    152
2011-02-11   

In [104]:
discrete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   state                      9134 non-null   object        
 1   response                   9134 non-null   object        
 2   coverage                   9134 non-null   object        
 3   education                  9134 non-null   object        
 4   effective_to_date          9134 non-null   datetime64[ns]
 5   employmentstatus           9134 non-null   object        
 6   gender                     9134 non-null   object        
 7   location_code              9134 non-null   object        
 8   marital_status             9134 non-null   object        
 9   months_since_last_claim    9134 non-null   int64         
 10  number_of_open_complaints  9134 non-null   int64         
 11  number_of_policies         9134 non-null   int64         
 12  policy

In [111]:
for column in discrete:
    print(discrete[column].agg(['nunique', 'count','dtypes']),'\n')

nunique         5
count        9134
dtypes     object
Name: state, dtype: object 

nunique         2
count        9134
dtypes     object
Name: response, dtype: object 

nunique         3
count        9134
dtypes     object
Name: coverage, dtype: object 

nunique         5
count        9134
dtypes     object
Name: education, dtype: object 

nunique                59
count                9134
dtypes     datetime64[ns]
Name: effective_to_date, dtype: object 

nunique         5
count        9134
dtypes     object
Name: employmentstatus, dtype: object 

nunique         2
count        9134
dtypes     object
Name: gender, dtype: object 

nunique         3
count        9134
dtypes     object
Name: location_code, dtype: object 

nunique         3
count        9134
dtypes     object
Name: marital_status, dtype: object 

nunique       36
count       9134
dtypes     int64
Name: months_since_last_claim, dtype: object 

nunique        6
count       9134
dtypes     int64
Name: number_of_open_complain

I don't see any kind of "strange" text in any of the variables. 
Thus it doesn't look to me like there's need for cleaning here

In [113]:
#Get categorical features.
cat_features = discrete.select_dtypes('object')
cat_features

,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,California,No,Basic,Bachelor,Employed,M,Urban,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car,Medsize
9130,California,Yes,Extended,College,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
9131,California,No,Extended,Bachelor,Unemployed,M,Suburban,Single,Corporate Auto,Corporate L2,Offer1,Branch,Four-Door Car,Medsize
9132,California,No,Extended,College,Employed,M,Suburban,Married,Personal Auto,Personal L2,Offer3,Branch,Four-Door Car,Large


In [116]:
#What should we do with the customer_id column?
## We can make it the index of the continuous dataframe
continuous = continuous.set_index('customer')
continuous

,customer_lifetime_value,income,monthly_premium_auto,months_since_policy_inception,total_claim_amount
customer,,,,,
BU79786,2763.519279,56274,69,5,384.811147
QZ44356,6979.535903,0,94,42,1131.464935
AI49188,12887.431650,48767,108,38,566.472247
WW63253,7645.861827,0,106,65,529.881344
HB64268,2813.692575,43836,73,44,138.130879
...,...,...,...,...,...
LA72316,23405.987980,71941,73,89,198.234764
PK87824,3096.511217,21604,79,28,379.200000
TD14365,8163.890428,0,85,37,790.784983
